## Method2

In [16]:
from utils import *

#### Load a file

In [17]:
mesh = pymesh.load_mesh("data/two_spheres.ply")

In [18]:
self_intersection_stats(mesh)

+-----------------------------------+---------+
| Metric                            |   Value |
+===================================+=========+
| Number of vertices                |     256 |
+-----------------------------------+---------+
| Number of faces                   |     504 |
+-----------------------------------+---------+
| Number of intersecting face pairs |      64 |
+-----------------------------------+---------+


In [19]:
visualize(mesh)

Widget(value='<iframe src="http://localhost:38155/index.html?ui=P_0x7fbc1a606f20_4&reconnect=auto" class="pyvi…

None

In [ ]:
visualize_intersection(mesh)

#### Get outer hull

In [20]:
outer_hull = pymesh.compute_outer_hull(mesh)

In [21]:
self_intersection_stats(outer_hull)

+-----------------------------------+---------+
| Metric                            |   Value |
+===================================+=========+
| Number of vertices                |     264 |
+-----------------------------------+---------+
| Number of faces                   |     524 |
+-----------------------------------+---------+
| Number of intersecting face pairs |       0 |
+-----------------------------------+---------+


#### Create a bounding box

In [89]:
local_min = np.array([ -0.90, -0.80, -0.90])
local_max = np.array([ 0.90,  0.20,  0.9])

In [90]:
submesh, face_mask = extract_submesh_by_bbox(outer_hull, local_min, local_max)

In [91]:
visualize(submesh, filename="Submesh")

Widget(value='<iframe src="http://localhost:38155/index.html?ui=P_0x7fbaf614a7a0_26&reconnect=auto" class="pyv…

None

In [92]:
remaining_mesh = extract_remaining_mesh(outer_hull, face_mask)

In [93]:
visualize(remaining_mesh, filename="remaining_mesh")

Widget(value='<iframe src="http://localhost:38155/index.html?ui=P_0x7fbaec119690_27&reconnect=auto" class="pyv…

None

#### Repair (or Remesh) the submesh
- `pymesh.collapse_short_edges(mesh, abs_threshold, rel_threshold, preserve_feature)`<br>
Collapses all edges with length less than a user specified threshold.

- `pymesh.split_long_edges(mesh, max_edge_length)`<br>
Splits long edges into 2 or more shorter edges.

- `pymesh.remove_obtuse_triangles(mesh, max_angle, max_iterations)`<br>
Splits each obtuse triangle into 2 or more right or sharp triangles.

- `pymesh.remove_degenerated_triangles(mesh, num_iterations)` <br>
Degenerate triangles are triangles with collinear vertices (i.e., some vertices are lying in the same straight line). They have zero areas and their normals are undefined.

In [94]:
from numpy.linalg import norm

def method2(mesh, detail=2e-2):

    bbox_min, bbox_max = mesh.bbox
    diag_len = norm(bbox_max - bbox_min)
    target_len = diag_len * detail
    print("Target resolution: {} mm".format(target_len))

    count = 0
    mesh, __ = pymesh.remove_degenerated_triangles(mesh, 100)
    mesh, __ = pymesh.split_long_edges(mesh, target_len)
    num_vertices = mesh.num_vertices
    while True:
        mesh, __ = pymesh.collapse_short_edges(mesh, 1e-6) # Remove extremely small edges
        mesh, __ = pymesh.collapse_short_edges(mesh, target_len,
                                               preserve_feature=True)
        mesh, __ = pymesh.remove_obtuse_triangles(mesh, 150.0, 100)
        if mesh.num_vertices == num_vertices:
            break

        num_vertices = mesh.num_vertices
        print("#v: {}".format(num_vertices))
        count += 1
        if count > 10: break

    mesh = pymesh.resolve_self_intersection(mesh)
    mesh, __ = pymesh.remove_duplicated_faces(mesh)
    mesh = pymesh.compute_outer_hull(mesh)
    mesh, __ = pymesh.remove_duplicated_faces(mesh)
    mesh, __ = pymesh.remove_obtuse_triangles(mesh, 179.0, 5)
    mesh, __ = pymesh.remove_isolated_vertices(mesh)

    return mesh

In [95]:
repaired_submesh = method2(submesh)

Target resolution: 0.06085787560109866 mm
#v: 922
#v: 917


In [96]:
visualize(repaired_submesh, filename="Repaired_submesh")

Widget(value='<iframe src="http://localhost:38155/index.html?ui=P_0x7fbaec119d20_28&reconnect=auto" class="pyv…

None

#### Align submesh
Since we have modified the submesh, we need to fix its boundary.

In [97]:
aligned_submesh = align_submesh_boundary(remaining_mesh, repaired_submesh)

In [98]:
visualize(aligned_submesh, filename="Method2")

Widget(value='<iframe src="http://localhost:38155/index.html?ui=P_0x7fbac8d60130_29&reconnect=auto" class="pyv…

None

#### Merge two meshes

In [99]:
repaired_full = replace_submesh_in_original(remaining_mesh, aligned_submesh)

#### Final refinement

In [100]:
final = refinement(repaired_full)

In [101]:
self_intersection_stats(final)

+-----------------------------------+---------+
| Metric                            |   Value |
+===================================+=========+
| Number of vertices                |     890 |
+-----------------------------------+---------+
| Number of faces                   |    1778 |
+-----------------------------------+---------+
| Number of intersecting face pairs |       0 |
+-----------------------------------+---------+


In [102]:
visualize(final, filename="Final Outerhull")

Widget(value='<iframe src="http://localhost:38155/index.html?ui=P_0x7fbac8d62170_30&reconnect=auto" class="pyv…

None

In [103]:
evaluation(outer_hull, final)

+-----------------------------------+-----------+------------+
| Metric                            |    Before |      After |
+===================================+===========+============+
| Number of vertices                | 264       |  890       |
+-----------------------------------+-----------+------------+
| Number of faces                   | 524       | 1778       |
+-----------------------------------+-----------+------------+
| Number of intersecting face pairs |   0       |    0       |
+-----------------------------------+-----------+------------+
| Volume                            |   6.10774 |    6.10364 |
+-----------------------------------+-----------+------------+
| Area                              |  17.6163  |   17.6158  |
+-----------------------------------+-----------+------------+


In [104]:
evaluate_displacement(outer_hull, final)

Max Vertex Displacement: 0.230974936710108
Mean Vertex Displacement: 0.07147648726846435


In [105]:
pymesh.save_mesh("final.ply", final, ascii=True)